<a href="https://colab.research.google.com/github/katrina906/CS6120-Summarization-Project/blob/main/text_rank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install rouge-score

In [2]:
import os
import pandas as pd
import numpy as np
import pickle
import string
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction import DictVectorizer
from collections import Counter, OrderedDict
from sklearn.metrics.pairwise import cosine_similarity
import networkx as nx
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# load data
df = pd.read_pickle("/content/drive/MyDrive/data/cleaned_df.pkl")
doc = df.iloc[0].sentences

In [ ]:
# TODO additional data cleaning when generating sentence simiarlity
# but print original sentence with puncutation etc. when display

In [5]:
# bag of words for each sentence in document
# TODO: other representations? tfidf, embeddings etc. 
def bag_of_words(doc):

  # for each sentence, split into list of words
  words = [sentence.split(' ') for sentence in doc]

  # bag of words
  vec = DictVectorizer()
  bow = vec.fit_transform(Counter(f) for f in words)

  return bow

In [6]:
# TODO: other similarity metrics?
def pagerank(bow):
  # similarity matrix between sentences
  sim =  cosine_similarity(bow)
  # graph where node = sentence, edge weight = simialarity score
  G = nx.from_numpy_array(sim)
  # page rank
  pr = nx.pagerank(G)

  return pr

In [7]:
def extract_summary(pr, topn):
  # sort keys in order
  bestkeys = sorted(pr, key=pr.get, reverse=True)[0:topn]
  return [doc[i] for i in bestkeys]

### Evaluation 
ROUGE metric:
https://kavita-ganesan.com/what-is-rouge-and-how-it-works-for-evaluation-of-summaries/#.YEKJyI5KiUl   
- Precision = # overlapping ngrams / # total ngrams in produced summary 
  - Measure of junk. Did we produce a lot in the generated summary that is not in the actual summary?
  - Important if we don't manually set the length. The generated summary could be very long which causes good recall
- Recall = # overlapping ngrams / # total ngrams in label summary  
  - Did we get all the words in the actual summary?
- F1 = harmonic mean
- N-Gram vs. LCS. Do we care about order? Don't need it to measure fluency/proper syntax. But ordering of words can indicate phrases 

Cons: 
- Doesn't look at sentence structure --> doesn't apply here because using correct sentences
- Doesn't consider meaning -- same words could have different meaning   
  
Also considered BLEU, but only gives precision.     
https://towardsdatascience.com/evaluating-text-output-in-nlp-bleu-at-your-own-risk-e8609665a213

In [8]:
def evaluate(predicted_summary, actual_summary):
  # TODO: unigram, bigram etc. models for rouge? - do we care about the order of the words?
  scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer = False)
  rouge = scorer.score(''.join(predicted_summary), ''.join(actual_summary))

  return bleu, rouge

In [9]:
bow = bag_of_words(doc)
pr = pagerank(bow)
predicted_summary = extract_summary(pr, 3) # TODO: choose best number of sentences (iterate with validation; rule of thumb based on EDA)
actual_summary = df.iloc[0].sentences_summary
bleu, rouge = evaluate(predicted_summary, actual_summary)

In [ ]:
rouge

{'rouge1': Score(precision=0.5, recall=0.19548872180451127, fmeasure=0.2810810810810811),
 'rouge2': Score(precision=0.23529411764705882, recall=0.09090909090909091, fmeasure=0.13114754098360656),
 'rougeL': Score(precision=0.34615384615384615, recall=0.13533834586466165, fmeasure=0.1945945945945946)}

In [ ]:
# TODO evaluate
# compare evaluations between models/configurations with paired bootstrap test? 